## Monthly Budget Notebook
#### Hi, thanks for taking a look at my budgeting notebook. This process used to be a lot simpler when I used a service called Empower and their "Personal Dashboard" for transaction aggregation. Then I started to notice some discrepancies and began to trust them less. After that, for some reason, their connectivity to Venmo went down the drain and as someone who uses Venmo a not insubstantial amount, it was leaving me with an incomplete picture of my monthly finances. I submitted support tickets but to no avail. 

#### So I figured this would be a useful project and at least then if I noticed things going wrong it'd be on _me_ and I'd be able to fix them _myself_.

##### _NB: If you want to use this for your own budgeting purposes, feel free!_

##### _Obvioulsy be careful and do so **at your own risk**. This is only set up right now for my own financial institutions that I use and has specific categorizations that I find helpful. The basic set-up for this notebook to work well is to have the notebook sitting in a directory (I have a structure that looks like this_
##### _`~/home/jake/Budget/2025/2025-09`_
##### _) and then have a sub directory called `Statements` which itself has a directory for each financial instituion that you have gathered statements from(`AmEx`,`Chase`,etc.)._

In [ ]:
import pandas as pd
import numpy as np
import glob

In [ ]:
#Aggregating a given month's statements into DataFrames
amex_paths = glob.glob("Statements/AmEx/*.csv")
amex_df = pd.concat([pd.read_csv(file) for file in amex_paths])


chase_path = glob.glob("Statements/Chase/*.CSV")
chase_df = pd.read_csv(chase_path[0])

venmo_path = glob.glob("Statements/Venmo/*.csv")
venmo_df = pd.read_csv(venmo_path[0],header=2)

wells_fargo_path = glob.glob("Statements/Wells Fargo/*.csv")
wells_fargo_df = pd.read_csv(wells_fargo_path[0], header=None)

### These next few cells can be safely turned off after actual use. I'm preserving them here to help show the process

In [ ]:
#Getting a quick look at the df columns pt.1
print(f"AmEx Columns {amex_df.columns}")
print(f"Chase Columns{chase_df.columns}")
print(f"Venmo Columns{venmo_df.columns}")
print(f"Wells Fargo Columns{wells_fargo_df.columns}")

In [ ]:
#Getting a quick look at the df columns pt.2
print(amex_df.head())
print(chase_df.head())
print(venmo_df.head())
print(wells_fargo_df.head())

In [ ]:
#Reordering the columns and dropping the ones I don't want before concatinating all of the dfs together

#AmEx
amex_df = amex_df[(amex_df['Amount'] >= 0) | 
    amex_df['Description'].str.contains("credit", case=False)]
amex_df['Amount'] = amex_df['Amount'].apply(
    lambda x: f"+{abs(x):.2f}" if x <= 0 else f"{-1*x:.2f}")

#Chase
chase_df = chase_df.iloc[:,[0,1,2]]
chase_df = chase_df.rename(columns={"Details":"Date","Posting Date":"Description","Description":"Amount"})
chase_df.reset_index(drop = True,inplace=True)
chase_df['Amount'] = chase_df['Amount'].astype(float)
chase_df['Amount'] = chase_df['Amount'].apply(lambda x: f"+{x:.2f}" if x > 0 else f"{x:.2f}")

#Venmo
venmo_df = venmo_df.iloc[:,[2,5,6,7,8]]
venmo_df['Description'] = "FROM: "+ venmo_df["From"] + " TO: "+ venmo_df["To"] + " NOTE: " + venmo_df["Note"]
venmo_df['Date'] = pd.to_datetime(venmo_df['Datetime']).dt.strftime('%m/%d/%Y')
venmo_df = venmo_df.iloc[:,[-1,-2,-3]]
venmo_df.dropna(subset=['Description'],inplace=True)
venmo_df.rename(columns = {"Amount (total)":"Amount"},inplace=True)
venmo_df['Amount'] = venmo_df['Amount'].astype(str).str.replace(r'[\$\s,]', '', regex=True)

#Wells Fargo
wells_fargo_df = wells_fargo_df.iloc[:,[0,-1,1]]
wells_fargo_df.rename(columns={0:"Date",4:"Description",1:"Amount"},inplace = True)
wells_fargo_df["Amount"] = wells_fargo_df["Amount"].apply(
    lambda x: f"+{x:.2f}" if x>0 else x)

In [ ]:
#Concatinating the dataframes into one
master_df = pd.concat([amex_df,chase_df,venmo_df,wells_fargo_df])
master_df.sort_values(["Date"],inplace = True)
master_df.reset_index(drop=True,inplace=True)
master_df

In [ ]:
#Removing the Apple Pay prefix from payments
master_df["Description"] = master_df["Description"].str.replace("AplPay","")

In [ ]:
with pd.option_context('display.max_rows', None,
                       'display.max_columns', None,
                       'display.precision', 3,
                       ):
    print(master_df)

In [ ]:
master_df.insert(0,"Expense Type","")

In [ ]:
master_df.insert(0,"Expense","")

In [ ]:
expense_mapping = [
    {
        'patterns': ["Trader Joe's"],
        'Expense': 'Groceries',
        'Expense Type': 'Variable Need'
    },
    {
        'patterns': ["Sinclair Gas"],
        'Expense': 'Gas',
        'Expense Type': 'Variable Need'
    },
    {
        'patterns': ["Thrive"],
        'Expense': 'Work Food',
        'Expense Type': 'Variable Want'
    },
    {
        'patterns': ["Steam"],
        'Expense': 'Gaming',
        'Expense Type': 'Variable Want'
    },
    {
        'patterns': ["Lemonade Insurance"],
        'Expense': 'Apartment',
        'Expense Type': 'Fixed Need'
    },
    {
        'patterns': ["Uber Eats"],
        'Expense': 'Eating Out',
        'Expense Type': 'Variable Want'
    },
    {
        'patterns': ["Nayax"],
        'Expense': 'Travel',
        'Expense Type': 'Variable Need'
    },
    {
        'patterns':['Klt floral'],
        'Expense': 'Gifts',
        'Expense Type': 'Variable Want'
    },
    {
        'patterns':['Spectrum'],
        'Expense': 'Internet',
        'Expense Type': 'Fixed Need'
    },
    {
        'patterns':['Cost Plus Drugs'],
        'Expense': 'Medical',
        'Expense Type': 'Variable Need'
    },
    {
        'patterns':['Geico'],
        'Expense': 'Car',
        'Expense Type': 'Fixed Need'
    }
]



In [ ]:
# Apply the mappings
for mapping in expense_mapping:
    # Create a regex pattern by joining the list of patterns with '|'
    pattern = '|'.join(mapping['patterns'])
    # Create a mask for rows where Description matches the pattern
    mask = master_df['Description'].str.contains(pattern, case=False, na=False)
    # Update 'Expense' and 'Expense Type' columns where the mask is True
    master_df.loc[mask, 'Expense'] = mapping['Expense']
    master_df.loc[mask, 'Expense Type'] = mapping['Expense Type']

In [ ]:
master_df['Description']

In [ ]:
master_df.to_clipboard(index=False)

In [ ]:
print(master_df)